In [22]:
import gymnasium as gym
import gymnasium_robotics

# PyTorch
import torch

import os

# from collections import deque
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback

In [23]:
env_id = 'FrankaKitchen-v1'
task = 'kettle'
gym.register_envs(gymnasium_robotics)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_actions = 9

In [24]:
granularity = 4
flat_dim = 59 + 7
action_space_size = granularity*9+1
transform_action_space = np.linspace(start=-1.0, stop=1.0, num=granularity)

def transform_action_from_int(action: int):
    if isinstance(action, list) :
        return action
    array_action = np.zeros(9)
    if action > action_space_size-1 or action < 0:
        raise AssertionError("transform_action_from_int")
    if action == action_space_size-1:
        return array_action
    which_action = action % 9
    singular_action = transform_action_space[action // 9]
    array_action[which_action] = singular_action
    return array_action

def transform_action_to_int(action) -> int:
    if isinstance(action, np.int64) :
        return action
    if sum(action) == 0.0:
        return action_space_size-1
    for i in range(9):
        if action[i] != 0.0:
            value = action[i]
            closest_quantized = min(transform_action_space, key=lambda x:abs(x - value))
            closest_quantized_index = -1
            for j in range(len(transform_action_space)):
                if transform_action_space[j] == closest_quantized:
                    closest_quantized_index = j
            if closest_quantized_index == -1: raise AssertionError("transform_action_to_int index is -1")
            return i + 9 * closest_quantized_index
    raise AssertionError("transform_action_to_int shouldn't be here")

def flatten_observation(observation):
    if not isinstance(observation, dict):
        return observation
    achieved = observation['achieved_goal'][task].astype(np.float32)
    obs = observation['observation'].astype(np.float32)

    flat_obs = np.concatenate([achieved, obs], dtype=np.float32)
    return flat_obs

In [25]:
def custom_reward(observation):
    achieved = observation['achieved_goal'][task][0:4]
    desired = observation['desired_goal'][task][0:4]
    res = 1.0 - np.linalg.norm(achieved - desired)
    assert res <= 1.0 and res >= 0.0, "Reward out of range!"
    return res   


In [26]:
obs_low = np.full((flat_dim,), -1e10, dtype=np.float32)
obs_high = np.full((flat_dim,), 1e10, dtype=np.float32)

class FlattenDictWrapper(gym.Wrapper):    
    def __init__(self, env):
        super().__init__(env)
        self.keys = env.observation_space.spaces.keys()
        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, shape=(flat_dim,), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(n=action_space_size)

    def observation(self, observation):
        return flatten_observation(observation)
    
    def action(self, action):
        return transform_action_to_int(action)
    
    def step(self, action):
        transformed_action = transform_action_to_int(action)
        obs, reward, terminated, truncated, info = self.env.step(transformed_action)
        if reward == 0.0:
            reward = custom_reward(obs)
        obs = flatten_observation(obs)
        return obs, reward, terminated, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return flatten_observation(obs)
    
    
def make_env():
    env = gym.make(env_id, render_mode=None, tasks_to_complete=[task])  # Or your actual task
    env = FlattenDictWrapper(env)
    return env


In [27]:
n_training_envs = 16
env = DummyVecEnv([make_env]*n_training_envs)
eval_env = DummyVecEnv([make_env])

In [28]:
max_timesteps = 50000
exploration_fraction=0.95
run_name = f"dqn_{granularity}p_{max_timesteps}_reward_shaping_{int(exploration_fraction*100)}_"+task

In [29]:
model = DQN("MlpPolicy", env, device=device, exploration_fraction=exploration_fraction, learning_rate=0.001)

In [30]:
# model = DQN.load("dqn_3_10000_"+task)

In [31]:
eval_log_dir = os.path.join("eval_logs", run_name)
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                              log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                              n_eval_episodes=5, deterministic=True,
                              render=False)

In [32]:

model.learn(total_timesteps=max_timesteps, callback=eval_callback)

/home/pc/repos/rl/rl-value-based/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=496, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
New best mean reward!
Eval num_timesteps=992, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=1488, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=1984, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=2480, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=2976, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=3472, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=3968, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=4464, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=4960, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_timesteps=5456, episode_reward=167.52 +/- 0.00
Episode length: 280.00 +/- 0.00
Eval num_tim

In [33]:
model.save(run_name)

In [34]:
for i in range(10):
	env_eval = make_env()
	obs, _ = env_eval.reset()
	done = False
	ep_reward = 0

	while not done:
		action, _ = model.predict(obs, deterministic=True)
		obs, reward, terminated, truncated, _ = env_eval.step(transform_action_from_int(action))
		obs = flatten_observation(obs)
		done = terminated or truncated
		ep_reward += reward
	print(f"Episode reward: {ep_reward}")

Episode reward: 167.51982922244304
Episode reward: 167.51982922244306
Episode reward: 167.5198292224431
Episode reward: 167.5198292224431
Episode reward: 167.5198292224431
Episode reward: 167.51982922244306
Episode reward: 167.51982922244306
Episode reward: 167.51982922244304
Episode reward: 167.5198292224431
Episode reward: 167.5198292224431


In [35]:

env_ = gym.make(env_id, render_mode=None, tasks_to_complete=[task])

In [36]:
state = env_.reset()

In [37]:
state[0]

{'observation': array([ 1.48711058e-01, -1.76751755e+00,  1.84306545e+00, -2.47715988e+00,
         2.59674953e-01,  7.13106755e-01,  1.59510009e+00,  4.73113960e-02,
         3.76087782e-02, -4.77725274e-04,  6.46840369e-04,  2.66336739e-05,
         8.08748015e-04,  5.60853916e-04, -8.05000536e-04, -9.80934344e-04,
         5.83153126e-04, -9.08293820e-04, -2.19807747e-04, -5.05184171e-05,
         3.32462059e-05, -4.50442273e-05, -3.94169622e-06, -4.24391463e-05,
         6.26077947e-05,  4.03237042e-05,  4.62669503e-04, -2.26772798e-04,
        -4.66765698e-04, -6.44101652e-03, -1.81471533e-03,  1.12579119e-03,
        -2.69440620e-01,  3.50381496e-01,  1.61944719e+00,  9.99972244e-01,
         4.08331116e-03, -6.59475048e-03, -2.48649692e-04,  1.09479091e-06,
        -7.90423333e-07,  1.93835756e-06,  6.32993734e-07, -9.95662358e-07,
        -3.71688496e-07,  2.06505846e-06,  8.71959368e-07, -8.29948416e-07,
         1.67522481e-06,  1.83082687e-07, -4.50304545e-06,  1.92959564e-0

In [38]:
sample = state[0]
desired_xyz = sample['desired_goal']['kettle'][0:3]
obs_xyz = sample['observation'][32:35]
distance_for_kettle = desired_xyz - obs_xyz

In [39]:
distance_for_kettle

array([0.03944062, 0.3996185 , 0.00055281])